In [21]:
# -*- coding: utf-8 -*-

from geopandas import GeoDataFrame
from shapely.geometry import MultiPoint, Point
from movingpandas.trajectory import Trajectory
from movingpandas.trajectory_collection import TrajectoryCollection
from movingpandas.geometry_utils import mrr_diagonal
from movingpandas.trajectory_utils import convert_time_ranges_to_segments
from movingpandas.time_range_utils import TemporalRangeWithTrajId


In [22]:
import geopandas as gpd
import hvplot.pandas 
from datetime import datetime, timedelta

In [23]:
import pandas as pd
from fiona.crs import from_epsg

In [24]:
gpx = 'GPX/2023/304 to UL 2023-03-09 0811.gpx'
bus_stops = gpd.read_file("GPKG/stops_304_to_UL.gpkg")
journey_plot_title = '304 to UL 2023-03-09 08:11'
gdf = gpd.read_file(gpx, layer='track_points').set_index('time')
gdf.drop(columns=['magvar', 'geoidheight', 'name', 'cmt', 'desc',
       'src', 'link1_href', 'link1_text', 'link1_type', 'link2_href', 
       'link2_text', 'link2_type', 'sym', 'type', 'fix', 'sat', 'hdop', 'vdop',
       'pdop', 'ageofdgpsdata', 'dgpsid'], inplace=True) 
track = Trajectory(gdf, 1)
track.add_speed(overwrite="True")
track.add_distance(overwrite="True")
track.df['speed_kph'] = track.df['speed'] * 3.6

In [77]:
gdf.index

DatetimeIndex(['2023-03-09 08:11:41+00:00', '2023-03-09 08:11:42+00:00',
               '2023-03-09 08:11:43+00:00', '2023-03-09 08:11:44+00:00',
               '2023-03-09 08:11:45+00:00', '2023-03-09 08:11:46+00:00',
               '2023-03-09 08:11:47+00:00', '2023-03-09 08:11:48+00:00',
               '2023-03-09 08:11:49+00:00', '2023-03-09 08:11:50+00:00',
               ...
               '2023-03-09 09:32:24+00:00', '2023-03-09 09:32:25+00:00',
               '2023-03-09 09:32:26+00:00', '2023-03-09 09:32:27+00:00',
               '2023-03-09 09:32:28+00:00', '2023-03-09 09:32:29+00:00',
               '2023-03-09 09:32:31+00:00', '2023-03-09 09:32:32+00:00',
               '2023-03-09 09:32:33+00:00', '2023-03-09 09:32:34+00:00'],
              dtype='datetime64[ns, UTC]', name='time', length=4788, freq=None)

In [78]:
track.df.index

DatetimeIndex(['2023-03-09 08:11:41+00:00', '2023-03-09 08:11:42+00:00',
               '2023-03-09 08:11:43+00:00', '2023-03-09 08:11:44+00:00',
               '2023-03-09 08:11:45+00:00', '2023-03-09 08:11:46+00:00',
               '2023-03-09 08:11:47+00:00', '2023-03-09 08:11:48+00:00',
               '2023-03-09 08:11:49+00:00', '2023-03-09 08:11:50+00:00',
               ...
               '2023-03-09 09:32:24+00:00', '2023-03-09 09:32:25+00:00',
               '2023-03-09 09:32:26+00:00', '2023-03-09 09:32:27+00:00',
               '2023-03-09 09:32:28+00:00', '2023-03-09 09:32:29+00:00',
               '2023-03-09 09:32:31+00:00', '2023-03-09 09:32:32+00:00',
               '2023-03-09 09:32:33+00:00', '2023-03-09 09:32:34+00:00'],
              dtype='datetime64[ns, UTC]', name='time', length=4788, freq=None)

In [25]:
CRS_METRIC = from_epsg(31256)
CRS_LATLON = from_epsg(4326)

class Node:
    def __init__(
        self,
        x=0,
        y=0,
        year=1970,
        month=1,
        day=1,
        hour=0,
        minute=0,
        second=0,
        millisec=0,
        value=0,
    ):
        self.geometry = Point(x, y)
        self.t = datetime(year, month, day, hour, minute, second, millisec)
        self.value = value

    def to_dict(self):
        return {"geometry": self.geometry, "t": self.t, "value": self.value}


In [106]:
def make_traj(nodes, crs=CRS_METRIC, id=1, parent=None):
    nodes = [node.to_dict() for node in nodes]
    df = pd.DataFrame(nodes).set_index("t")
    gdf = GeoDataFrame(df)
    if crs:
        gdf = gdf.set_crs(crs=crs, allow_override=True)
    return Trajectory(gdf, id, parent=parent)

def make_traj2(nodes, crs=CRS_METRIC, id=1, parent=None):
    nodes = [node.to_dict() for node in nodes]
    df = pd.DataFrame(nodes)
    df["t"] = pd.to_datetime(df["t"], utc=True)
    df = df.set_index("t")
    gdf = GeoDataFrame(df)
    if crs:
        gdf = gdf.set_crs(crs=crs, allow_override=True)
    return Trajectory(gdf, id, parent=parent)


test_traj = make_traj(
            [
                Node(0, 0),
                Node(0, 10, second=1),
                Node(0, 20, second=2),
                Node(0, 21, second=4),
                Node(0, 22, second=6),
                Node(0, 30, second=8),
                Node(0, 40, second=10),
                Node(1, 50, second=15),
            ]
        )


test_traj2 = make_traj2(
            [
                Node(0, 0),
                Node(0, 10, second=1),
                Node(0, 20, second=2),
                Node(0, 21, second=4),
                Node(0, 22, second=6),
                Node(0, 30, second=8),
                Node(0, 40, second=10),
                Node(0, 40, second=13),
                Node(1, 50, second=15),
            ]
        )

In [27]:
    def _process_traj(traj, max_diameter, min_duration):
        detected_stops = []
        segment_geoms = []
        segment_times = []
        geom = MultiPoint()
        is_stopped = False
        previously_stopped = False

        for index, data in traj.df[traj.get_geom_column_name()].items():
            segment_geoms.append(data)
            geom = geom.union(data)
            segment_times.append(index)

            if not is_stopped:  # remove points to the specified min_duration
                while (
                    len(segment_geoms) > 2
                    and segment_times[-1] - segment_times[0] >= min_duration
                ):
                    segment_geoms.pop(0)
                    segment_times.pop(0)
                # after removing extra points, re-generate geometry
                geom = MultiPoint(segment_geoms)

            if (
                len(segment_geoms) > 1
                and mrr_diagonal(geom, traj.is_latlon) < max_diameter
            ):
                is_stopped = True
            else:
                is_stopped = False

            if len(segment_geoms) > 1:
                segment_end = segment_times[-2]
                segment_begin = segment_times[0]
                if not is_stopped and previously_stopped:
                    if (
                        segment_end - segment_begin >= min_duration
                    ):  # detected end of a stop
                        detected_stops.append(
                            TemporalRangeWithTrajId(segment_begin, segment_end, traj.id)
                        )
                        segment_geoms = []
                        segment_times = []
                        geom = MultiPoint()

            previously_stopped = is_stopped

        if is_stopped and segment_times[-1] - segment_times[0] >= min_duration:
            detected_stops.append(
                TemporalRangeWithTrajId(segment_times[0], segment_times[-1], traj.id)
            )

        return detected_stops


In [28]:
    def get_stop_time_ranges(traj, max_diameter, min_duration):
        """
        Returns detected stop start and end times

        Parameters
        ----------
        max_diameter : float
            Maximum diameter for stop detection
        min_duration : datetime.timedelta
            Minimum stop duration

        Returns
        -------
        list
            TemporalRanges of detected stops
        """
        if isinstance(traj, Trajectory):
            return _process_traj(traj, max_diameter, min_duration)
        else:
            raise TypeError


In [79]:
test_traj

Trajectory 1 (1970-01-01 00:00:00 to 1970-01-01 00:00:15) | Size: 8 | Length: 50.0m
Bounds: (0.0, 0.0, 1.0, 50.0)
LINESTRING (0 0, 0 10, 0 20, 0 21, 0 22, 0 30, 0 40, 1 50)

In [107]:
test_traj2

Trajectory 1 (1970-01-01 00:00:00+00:00 to 1970-01-01 00:00:15+00:00) | Size: 9 | Length: 50.0m
Bounds: (0.0, 0.0, 1.0, 50.0)
LINESTRING (0 0, 0 10, 0 20, 0 21, 0 22, 0 30, 0 40, 0 40, 1 50)

In [80]:
test_traj.df.index

DatetimeIndex(['1970-01-01 00:00:00', '1970-01-01 00:00:01',
               '1970-01-01 00:00:02', '1970-01-01 00:00:04',
               '1970-01-01 00:00:06', '1970-01-01 00:00:08',
               '1970-01-01 00:00:10', '1970-01-01 00:00:15'],
              dtype='datetime64[ns]', name='t', freq=None)

In [108]:
test_traj2.df.index

DatetimeIndex(['1970-01-01 00:00:00+00:00', '1970-01-01 00:00:01+00:00',
               '1970-01-01 00:00:02+00:00', '1970-01-01 00:00:04+00:00',
               '1970-01-01 00:00:06+00:00', '1970-01-01 00:00:08+00:00',
               '1970-01-01 00:00:10+00:00', '1970-01-01 00:00:13+00:00',
               '1970-01-01 00:00:15+00:00'],
              dtype='datetime64[ns, UTC]', name='t', freq=None)

In [31]:
max_diameter = 3

In [32]:
min_duration=timedelta(seconds=2)

In [33]:
stop_time_ranges = get_stop_time_ranges(track, max_diameter, min_duration)

In [73]:
stop_time_ranges

In [34]:
stop_time_ranges2 = get_stop_time_ranges(test_traj, max_diameter, min_duration)

In [74]:
stop_time_ranges2

In [118]:
stop_time_ranges3 = get_stop_time_ranges(test_traj2, max_diameter, min_duration)

In [119]:
stop_time_ranges3

In [10]:
stops = TrajectoryCollection(
    convert_time_ranges_to_segments(track, stop_time_ranges)
)

In [35]:
stops2 = TrajectoryCollection(
    convert_time_ranges_to_segments(test_traj, stop_time_ranges2)
)

In [120]:
stops3 = TrajectoryCollection(
    convert_time_ranges_to_segments(test_traj2, stop_time_ranges3)
)

In [75]:
stops

TrajectoryCollection with 59 trajectories

In [99]:
for traj in stops:
    print(traj.df.index.dtype)

datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]
datetime64[ns, UTC]


In [76]:
stops2

TrajectoryCollection with 1 trajectories

In [114]:
for traj in stops2:
    print(traj.df.index.dtype)

datetime64[ns]


In [121]:
stops3

TrajectoryCollection with 2 trajectories

In [122]:
for traj in stops3:
    print(traj.df.index.dtype)

datetime64[ns, UTC]
datetime64[ns, UTC]


In [36]:
stop_pts = GeoDataFrame(columns=["geometry"]).set_geometry("geometry")

In [37]:
stop_pts2 = GeoDataFrame(columns=["geometry"]).set_geometry("geometry")

In [123]:
stop_pts3 = GeoDataFrame(columns=["geometry"]).set_geometry("geometry")

In [38]:
stop_pts["stop_id"] = [track.id for track in stops.trajectories]

In [39]:
stop_pts2["stop_id"] = [track.id for track in stops2.trajectories]

In [124]:
stop_pts3["stop_id"] = [track.id for track in stops3.trajectories]

In [40]:
stop_pts = stop_pts.set_index("stop_id")

In [41]:
stop_pts2 = stop_pts2.set_index("stop_id")

In [125]:
stop_pts3 = stop_pts3.set_index("stop_id")

In [136]:
    def get_start_time_numpy(self):
        """
        Return the trajectory's start time.

        Returns
        -------
        datetime.datetime
            Trajectory start time
        """
        return traj.df.index.min()

In [137]:
stop_pts4 = stop_pts
for stop in stops:
    stop_pts4.at[stop.id, "start_time"] = get_start_time_numpy(stop)
    stop_pts4.at[stop.id, "end_time"] = stop.get_end_time()
    pt = Point(stop.df.geometry.x.median(), stop.df.geometry.y.median())
    stop_pts4.at[stop.id, "geometry"] = pt
    stop_pts4.at[stop.id, "traj_id"] = stop.parent.id

In [42]:
for stop in stops:
    stop_pts.at[stop.id, "start_time"] = stop.get_start_time()
    stop_pts.at[stop.id, "end_time"] = stop.get_end_time()
    pt = Point(stop.df.geometry.x.median(), stop.df.geometry.y.median())
    stop_pts.at[stop.id, "geometry"] = pt
    stop_pts.at[stop.id, "traj_id"] = stop.parent.id

In [43]:
for stop in stops2:
    stop_pts2.at[stop.id, "start_time"] = stop.get_start_time()
    stop_pts2.at[stop.id, "end_time"] = stop.get_end_time()
    pt = Point(stop.df.geometry.x.median(), stop.df.geometry.y.median())
    stop_pts2.at[stop.id, "geometry"] = pt
    stop_pts2.at[stop.id, "traj_id"] = stop.parent.id

In [126]:
for stop in stops3:
    stop_pts3.at[stop.id, "start_time"] = stop.get_start_time()
    stop_pts3.at[stop.id, "end_time"] = stop.get_end_time()
    pt = Point(stop.df.geometry.x.median(), stop.df.geometry.y.median())
    stop_pts3.at[stop.id, "geometry"] = pt
    stop_pts3.at[stop.id, "traj_id"] = stop.parent.id

In [138]:
stop_pts4["start_time"].apply(type)

stop_id
1_2023-03-09 08:11:41+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:25:47+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:34:20+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:34:54+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:38:07+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:38:34+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:39:03+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:39:43+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:40:42+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:41:51+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:43:14+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:44:04+00:00    <class 'pandas._libs.tslibs.timestamps.Timesta...
1_2023-03-09 08:44:4

In [127]:
stop_pts3["start_time"].apply(type)

stop_id
1_1970-01-01 00:00:02+00:00    <class 'datetime.datetime'>
1_1970-01-01 00:00:10+00:00    <class 'datetime.datetime'>
Name: start_time, dtype: object

In [100]:
stop_pts["start_time"].apply(type)

stop_id
1_2023-03-09 08:11:41+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:25:47+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:34:20+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:34:54+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:38:07+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:38:34+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:39:03+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:39:43+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:40:42+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:41:51+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:43:14+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:44:04+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:44:48+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:45:33+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:46:16+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:46:57+00:00    <class 'datetime.datetime'>
1_2023-03-09 08:47:25+00:00    <class 'datetime.

In [44]:
stop_pts

,geometry,start_time,end_time,traj_id
stop_id,,,,
1_2023-03-09 08:11:41+00:00,POINT (-8.65169 52.64003),2023-03-09 08:11:41+00:00,2023-03-09 08:25:45+00:00,1.0
1_2023-03-09 08:25:47+00:00,POINT (-8.65173 52.64003),2023-03-09 08:25:47+00:00,2023-03-09 08:33:45+00:00,1.0
1_2023-03-09 08:34:20+00:00,POINT (-8.64920 52.64146),2023-03-09 08:34:20+00:00,2023-03-09 08:34:40+00:00,1.0
1_2023-03-09 08:34:54+00:00,POINT (-8.64815 52.64200),2023-03-09 08:34:54+00:00,2023-03-09 08:37:38+00:00,1.0
1_2023-03-09 08:38:07+00:00,POINT (-8.64700 52.64275),2023-03-09 08:38:07+00:00,2023-03-09 08:38:32+00:00,1.0
1_2023-03-09 08:38:34+00:00,POINT (-8.64678 52.64297),2023-03-09 08:38:34+00:00,2023-03-09 08:39:01+00:00,1.0
1_2023-03-09 08:39:03+00:00,POINT (-8.64660 52.64319),2023-03-09 08:39:03+00:00,2023-03-09 08:39:41+00:00,1.0
1_2023-03-09 08:39:43+00:00,POINT (-8.64633 52.64357),2023-03-09 08:39:43+00:00,2023-03-09 08:40:40+00:00,1.0
1_2023-03-09 08:40:42+00:00,POINT (-8.64626 52.64370),2023-03-09 08:40:42+00:00,2023-03-09 08:41:32+00:00,1.0


In [45]:
stop_pts2

,geometry,start_time,end_time,traj_id
stop_id,,,,
1_1970-01-01 00:00:02,POINT (0.00000 21.00000),1970-01-01 00:00:02,1970-01-01 00:00:06,1.0


In [117]:
stop_pts3

,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
1_1970-01-01 00:00:02,POINT (0.00000 21.00000),1970-01-01 00:00:02,1970-01-01 00:00:06,1.0,0 days 00:00:04
1_1970-01-01 00:00:10,POINT (0.00000 40.00000),1970-01-01 00:00:10,1970-01-01 00:00:13,1.0,0 days 00:00:03


In [46]:
stop_pts["duration_s"] = stop_pts["end_time"] - stop_pts["start_time"]

In [47]:
stop_pts2["duration_s"] = stop_pts2["end_time"] - stop_pts2["start_time"]

In [95]:
stop_pts3["duration_s"] = stop_pts3["end_time"] - stop_pts3["start_time"]

In [48]:
stop_pts.dtypes

geometry      geometry
start_time      object
end_time        object
traj_id        float64
duration_s      object
dtype: object

In [49]:
stop_pts2.dtypes

geometry             geometry
start_time     datetime64[ns]
end_time       datetime64[ns]
traj_id               float64
duration_s    timedelta64[ns]
dtype: object

In [96]:
stop_pts3.dtypes

geometry             geometry
start_time     datetime64[ns]
end_time       datetime64[ns]
traj_id               float64
duration_s    timedelta64[ns]
dtype: object

In [61]:
stop_pts.iat[0,1]

datetime.datetime(2023, 3, 9, 8, 11, 41, tzinfo=datetime.timezone.utc)

In [62]:
print(stop_pts.iat[0,1])

2023-03-09 08:11:41+00:00


In [55]:
stop_pts.iat[0,2]

datetime.datetime(2023, 3, 9, 8, 25, 45, tzinfo=datetime.timezone.utc)

In [56]:
stop_pts.iat[0,4]

datetime.timedelta(seconds=844)

In [63]:
stop_pts2.iat[0,1]

Timestamp('1970-01-01 00:00:02')

In [64]:
print(stop_pts2.iat[0,1])

1970-01-01 00:00:02


In [58]:
stop_pts2.iat[0,2]

Timestamp('1970-01-01 00:00:06')

In [59]:
stop_pts2.iat[0,4]

Timedelta('0 days 00:00:04')

In [52]:
stop_pts["duration_s"].apply(type)

stop_id
1_2023-03-09 08:11:41+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:25:47+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:34:20+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:34:54+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:38:07+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:38:34+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:39:03+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:39:43+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:40:42+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:41:51+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:43:14+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:44:04+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:44:48+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:45:33+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:46:16+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:46:57+00:00    <class 'datetime.timedelta'>
1_2023-03-09 08:47:25+00:00    <

In [ ]:
stop_pts2["duration_s"].apply(type)

In [66]:
stop_pts['duration_s'] = pd.to_datetime(stop_pts['duration_s']).dt.time

TypeError: <class 'datetime.timedelta'> is not convertible to datetime, at position 0

In [ ]:
stop_pts2['duration_s'] = pd.to_datetime(stop_pts2['duration_s']).dt.time

In [33]:
stop_pts.dtypes

geometry      geometry
start_time      object
end_time        object
traj_id        float64
duration_s      object
dtype: object

In [32]:
stop_pts

,geometry,start_time,end_time,traj_id,duration_s
stop_id,,,,,
1_2023-03-09 08:11:41+00:00,POINT (-8.65169 52.64003),2023-03-09 08:11:41+00:00,2023-03-09 08:25:45+00:00,1.0,00:14:04
1_2023-03-09 08:25:47+00:00,POINT (-8.65173 52.64003),2023-03-09 08:25:47+00:00,2023-03-09 08:33:45+00:00,1.0,00:07:58
1_2023-03-09 08:34:20+00:00,POINT (-8.64920 52.64146),2023-03-09 08:34:20+00:00,2023-03-09 08:34:40+00:00,1.0,00:00:20
1_2023-03-09 08:34:54+00:00,POINT (-8.64815 52.64200),2023-03-09 08:34:54+00:00,2023-03-09 08:37:38+00:00,1.0,00:02:44
1_2023-03-09 08:38:07+00:00,POINT (-8.64700 52.64275),2023-03-09 08:38:07+00:00,2023-03-09 08:38:32+00:00,1.0,00:00:25
1_2023-03-09 08:38:34+00:00,POINT (-8.64678 52.64297),2023-03-09 08:38:34+00:00,2023-03-09 08:39:01+00:00,1.0,00:00:27
1_2023-03-09 08:39:03+00:00,POINT (-8.64660 52.64319),2023-03-09 08:39:03+00:00,2023-03-09 08:39:41+00:00,1.0,00:00:38
1_2023-03-09 08:39:43+00:00,POINT (-8.64633 52.64357),2023-03-09 08:39:43+00:00,2023-03-09 08:40:40+00:00,1.0,00:00:57
1_2023-03-09 08:40:42+00:00,POINT (-8.64626 52.64370),2023-03-09 08:40:42+00:00,2023-03-09 08:41:32+00:00,1.0,00:00:50


In [70]:
stop_pts["duration_seconds"] = stop_pts["duration_s"].apply(lambda x: x.seconds)

In [71]:
stop_pts

,geometry,start_time,end_time,traj_id,duration_s,duration_seconds
stop_id,,,,,,
1_2023-03-09 08:11:41+00:00,POINT (-8.65169 52.64003),2023-03-09 08:11:41+00:00,2023-03-09 08:25:45+00:00,1.0,0:14:04,844
1_2023-03-09 08:25:47+00:00,POINT (-8.65173 52.64003),2023-03-09 08:25:47+00:00,2023-03-09 08:33:45+00:00,1.0,0:07:58,478
1_2023-03-09 08:34:20+00:00,POINT (-8.64920 52.64146),2023-03-09 08:34:20+00:00,2023-03-09 08:34:40+00:00,1.0,0:00:20,20
1_2023-03-09 08:34:54+00:00,POINT (-8.64815 52.64200),2023-03-09 08:34:54+00:00,2023-03-09 08:37:38+00:00,1.0,0:02:44,164
1_2023-03-09 08:38:07+00:00,POINT (-8.64700 52.64275),2023-03-09 08:38:07+00:00,2023-03-09 08:38:32+00:00,1.0,0:00:25,25
1_2023-03-09 08:38:34+00:00,POINT (-8.64678 52.64297),2023-03-09 08:38:34+00:00,2023-03-09 08:39:01+00:00,1.0,0:00:27,27
1_2023-03-09 08:39:03+00:00,POINT (-8.64660 52.64319),2023-03-09 08:39:03+00:00,2023-03-09 08:39:41+00:00,1.0,0:00:38,38
1_2023-03-09 08:39:43+00:00,POINT (-8.64633 52.64357),2023-03-09 08:39:43+00:00,2023-03-09 08:40:40+00:00,1.0,0:00:57,57
1_2023-03-09 08:40:42+00:00,POINT (-8.64626 52.64370),2023-03-09 08:40:42+00:00,2023-03-09 08:41:32+00:00,1.0,0:00:50,50


In [15]:
if len(stops) > 0:
    stop_pts["duration_s"] = (
        stop_pts["end_time"] - stop_pts["start_time"]
    ).dt.total_seconds()
    stop_pts["traj_id"] = stop_pts["traj_id"].astype(type(stop.parent.id))

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:









return stop_pts
